# Imports

In [1]:
import random
import os
import shutil

from IPython.display import display_html

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

import torch
print(torch.cuda.is_available())

from ultralytics import YOLO

np.random.seed(42)
DATA_PATH = "E:\\OU\\master\\rm\\pre_shot_sniper_detection\\data\\data.yaml" #must be absolute path

True


# Functions

In [2]:
def remove_cache():
    for directory in ["train", "test", "valid"]:
        path = f"./data/{directory}/labels.cache"
        if os.path.isfile(path):
            os.remove(path)

In [3]:
def select_imgs(num_imgs:int):
    remove_cache()
    
    num_train = int(num_imgs *0.8)
    num_val = int(num_imgs *0.1)
    
    img_path = "./data/raw/images/"
    bb_path = "./data/raw/labels/"
    
    files = os.listdir(img_path)[:num_imgs]

    
    for file in files[:num_train]:
        img_file = file[:-3]+"jpg" 
        bb_file = file[:-3]+"txt"
        shutil.copy(img_path+img_file, f"./data/train/images/{img_file}")
        shutil.copy(bb_path+bb_file, f"./data/train/labels/{bb_file}")
        
    for file in files[num_train:num_train+num_val]:
        img_file = file[:-3]+"jpg" 
        bb_file = file[:-3]+"txt"
        shutil.copy(img_path+img_file, f"./data/valid/images/{img_file}")
        shutil.copy(bb_path+bb_file, f"./data/valid/labels/{bb_file}")
    
    for file in files[num_train+num_val:]:
        img_file = file[:-3]+"jpg" 
        bb_file = file[:-3]+"txt"
        shutil.copy(img_path+img_file, f"./data/test/images/{img_file}")
        shutil.copy(bb_path+bb_file, f"./data/test/labels/{bb_file}")

In [ ]:
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [4]:
def train(name:str):
    model = YOLO("./yolov8n.pt")
    model.train(data=DATA_PATH, epochs=100, imgsz=640, name=name, device=0, project="normal")
    model.val(name=name+"_val") #conf is 0.001
    del model
    torch.cuda.empty_cache()